In [ ]:
from VR_Trajectory_analysis import *

In [ ]:
directory = '/Users/apaula/ownCloud/MatrexVR1/20250403_ColorGeometry_Data/RunData'

In [ ]:
df1 = get_combined_df(directory, trim_seconds=1.0)

In [ ]:
directory = '/Users/apaula/ownCloud/MatrexVR1/20250507_ColorGeometryV2_Data/RunData'

In [ ]:
df2 = get_combined_df(directory, trim_seconds=1.0)

In [ ]:
directory = '/Users/apaula/ownCloud/MatrexVR1/20250520_Geometry_V3/20250521_ColorGeometry_V3_Data/RunData'

In [ ]:
df3 = get_combined_df(directory, trim_seconds=1.0)
df3['ConfigFile'] = df3['stepName']

In [ ]:
df = pd.concat([df1, df2, df3], ignore_index=True)

In [ ]:
df["FlyID"].nunique()

In [ ]:
df.columns

In [ ]:
df = add_trial_id_and_displacement(df)
df = add_trial_time(df)

In [ ]:
df


In [ ]:
df_stationary, df_normal, df_excessive, stationary_ids, normal_ids, excessive_ids = classify_trials_by_displacement(df[df['Scene'].isin(['Choice_noBG', 'Choice_desync'])], min_disp=0, max_disp=500)

In [ ]:
plot_trajectories(df_normal, 'normal')

In [ ]:
import pandas as pd
import numpy as np
import re

def parse_angle_from_config(config_filename: str, default_angle: float = 20.0) -> float:
    """
    Extracts the 'XX' in e.g. "constantSize_XXdeg" from the config filename.
    If no match is found (deg info is missing), returns default_angle (usually 20°).
    """
    pattern = r"_([0-9]+)deg"  # looks for something like "_30deg_"
    match = re.search(pattern, config_filename)
    if match:
        return float(match.group(1))
    else:
        return default_angle

def get_left_right_goals(angle_deg: float, radius: float = 60.0):
    """
    Given a total separation angle (in degrees) and a circle radius, 
    return (x,z) for the left goal and right goal, respectively,
    placed symmetrically about the positive z-axis.
    """
    # Convert half the separation angle to radians
    half_angle_rad = np.radians(angle_deg / 2.0)

    # By convention here:
    #  - "left" is at -half_angle_rad
    #  - "right" is at +half_angle_rad
    #
    # Since 0° is along +z, we do:
    #   x = r * sin(theta)
    #   z = r * cos(theta)
    #
    # left uses -half_angle_rad; right uses +half_angle_rad
    left_x = radius * np.sin(-half_angle_rad)
    left_z = radius * np.cos(-half_angle_rad)
    right_x = radius * np.sin(half_angle_rad)
    right_z = radius * np.cos(half_angle_rad)

    return (left_x, left_z), (right_x, right_z)

def get_first_goal_reached(
    df_normal,
    goals,            # A list of (goal_name, (x, z)) tuples
    threshold=7
):
    """
    Given a subset of data for a single config, determines the first 
    goal reached for each UniqueTrialID (unchanged from your snippet).
    """
    def distance(p1, p2):
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
    
    results = []
    
    # Group by UniqueTrialID
    for trial_id, trial_data in df_normal.groupby('UniqueTrialID'):
        config = trial_data['ConfigFile'].iloc[0]
        
        # Sort by time
        trial_data = trial_data.sort_values(by='trial_time')
        
        first_reached = None
        reached_time = None
        
        for idx, row in trial_data.iterrows():
            participant_pos = (row['GameObjectPosX'], row['GameObjectPosZ'])
            # Check each goal
            for goal_name, goal_pos in goals:
                if distance(participant_pos, goal_pos) <= threshold:
                    first_reached = goal_name
                    reached_time = row['trial_time']
                    break
            if first_reached is not None:
                break
        
        results.append((trial_id, config, first_reached, reached_time))
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results, columns=[
        'UniqueTrialID',
        'ConfigFile',
        'FirstReachedGoal',
        'GoalReachedTime'
    ])
    
    return results_df


def analyze_variable_angle_configs(df_normal, threshold=3.5, default_angle=20.0):
    """
    1) Finds unique config files that start with "BinaryChoice_constantSize" and 
       end with "BlackCylinder_BlackCylinder.json".
    2) Parses the angle, sets default to 20° if missing.
    3) Computes left/right goals for each angle.
    4) Calls get_first_goal_reached for each subset, concatenates results.
    """
    # Filter only the configs we care about
    mask = df_normal['ConfigFile'].str.contains(
        r'BinaryChoice|ScalingChoice',   # match either substring
        regex=True,     # ← default is True, but explicit is fine
        na=False        # avoid NaN -> False
    )
    df_subset = df_normal[mask]
    
    unique_configs = df_subset['ConfigFile'].unique()
    
    all_results = []
    
    for cfg in unique_configs:
        angle_deg = parse_angle_from_config(cfg, default_angle=default_angle)
        (left_goal, right_goal) = get_left_right_goals(angle_deg, radius=60)
        
        # Subset data for this particular config
        df_config = df_subset[df_subset['ConfigFile'] == cfg].copy()
        
        # Determine the first reached goal
        res = get_first_goal_reached(
            df_config,
            goals=[("left", left_goal), ("right", right_goal)],
            threshold=threshold
        )
        all_results.append(res)
    
    if all_results:
        return pd.concat(all_results, ignore_index=True)
    else:
        # No matching config files found
        return pd.DataFrame(columns=['UniqueTrialID', 'ConfigFile', 'FirstReachedGoal', 'GoalReachedTime'])

# -------------------------------------------------------------------
# Example Usage:
final_results = analyze_variable_angle_configs(df_normal, threshold=3.5, default_angle=20.0)
#
# final_results will have one row per UniqueTrialID, giving you
# the earliest goal reached and its time, for each of the "BinaryChoice_constantSize_..._BlackCylinder_BlackCylinder.json"
# config files in df_normal.
# -------------------------------------------------------------------


In [ ]:
df_normal["FlyID"].nunique()

In [ ]:
final_results['GoalReachedTime'].notna().sum()

In [ ]:
all_results=final_results

In [ ]:
all_results.to_pickle("geometry_results_df_3.pkl")

In [ ]:
results_df = pd.read_pickle("geometry_results_df_3.pkl")

In [ ]:
results_df=all_results


In [ ]:
df

In [ ]:
# 1. Keep only trials that actually reached a goal
valid_results = results_df.dropna(subset=['FirstReachedGoal'])

# 2. Merge the cutoff times back into df
#    We merge on 'UniqueTrialID' to get each trial's GoalReachedTime.
df_merged = pd.merge(df, valid_results[['UniqueTrialID', 'GoalReachedTime', 'FirstReachedGoal']], on='UniqueTrialID', how='inner')

# 3. Filter df so that only rows with trial_time less than or equal to the goal time are kept
df_cut = df_merged[df_merged['trial_time'] <= df_merged['GoalReachedTime']]

In [ ]:
df_merged['GoalReachedTime'].max()

In [ ]:
# Save DataFrames as pickle (binary, preserves dtypes & indexes)
df_cut_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/color_bias_geometry3/df_cut.pkl'
results_df_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/color_bias_geometry3/results_df.pkl'
df_cut.to_pickle(df_cut_path_pickle)
results_df.to_pickle(results_df_path_pickle)

In [ ]:
# Script can be run from here if raw data hasn't changed
# This cell marks the point where you can start running the script if the raw data remains unchanged. All necessary preprocessed data has been saved and can be loaded directly from the pickle files.

In [ ]:
from VR_Trajectory_analysis import *
df_cut_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/color_bias_geometry3/df_cut.pkl'
results_df_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/color_bias_geometry3/results_df.pkl'
df_cut = pd.read_pickle(df_cut_path_pickle)
results_df = pd.read_pickle(results_df_path_pickle)

In [ ]:
df_cut["FlyID"].nunique()

In [ ]:
df_cut.columns

In [ ]:
df_joined = pd.merge(
    df_cut,
    results_df[['UniqueTrialID', 'FirstReachedGoal']],
    on='UniqueTrialID',
    how='left'
)
#add angle column to df_joined
df_joined['Angle'] = df_joined['ConfigFile'].apply(lambda x: parse_angle_from_config(x, default_angle=999.0))

In [ ]:
df_joined['Angle'].unique()

In [ ]:
# -----------------------------------------------------------------
#  Trajectories for all  "BlueCylinder_BlueGreenCylinder" configs
#  shown in a grid, one panel per Angle
# -----------------------------------------------------------------
import math, re
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec

# 1.  The joined dataframe is assumed to exist:  df_joined

# 2.  Keep only the configurations we care about ---------------------------
accept_tags = ("BlueCylinder_BlueGreenCylinder",
               "Blue_BlueGreen")

df_filt = df_joined[df_joined['ConfigFile'].str.contains('|'.join(accept_tags))]

# make sure Angle is numeric so the sort works
df_filt['Angle'] = pd.to_numeric(df_filt['Angle'], errors='coerce')

angles = sorted(df_filt['Angle'].dropna().unique())
if not angles:
    raise ValueError("No matching configs with an Angle column found.")

# 3.  Build grid layout -----------------------------------------------------
n_panels = len(angles)
n_cols   = 3                               # change to taste
n_rows   = math.ceil(n_panels / n_cols)

fig = plt.figure(figsize=(n_cols*4, n_rows*3.75))
gs  = gridspec.GridSpec(n_rows, n_cols, wspace=.25, hspace=.35)

# 4.  One sub-plot per Angle -------------------------------------------------
for idx, angle in enumerate(angles):
    ax = fig.add_subplot(gs[idx])
    g  = df_filt[df_filt['Angle'] == angle]

    # trajectories
    for trial_id, td in g.groupby('UniqueTrialID'):
        ax.plot(td['GameObjectPosX'], td['GameObjectPosZ'], alpha=.1)

    # basic aesthetics
    ax.set_xlim(-40, 40)
    ax.set_ylim(-10, 70)
    ax.set_aspect('equal', adjustable='box')

    ax.set_xticks([-60, -40, -20, 0, 20, 40, 60])
    ax.set_yticks([0, 20, 40, 60])
    ax.tick_params(labelsize=8)

    ax.set_title(f"{int(angle)}°", fontsize=10)

    # left / right counts & ratios
    res   = results_df.loc[results_df['UniqueTrialID'].isin(g['UniqueTrialID'])]
    left  = (res['FirstReachedGoal'] == 'left').sum()
    right = (res['FirstReachedGoal'] == 'right').sum()
    tot   = left + right
    txt   = f"L: {left} ({left/tot:.2f})\nR: {right} ({right/tot:.2f})" if tot else "no goals"

    ax.text(.02, .98, txt, transform=ax.transAxes, va='top',
            fontsize=8, bbox=dict(boxstyle="round", fc="w", ec=".5"))

# hide any leftover empty cells
for j in range(idx+1, n_rows*n_cols):
    fig.add_subplot(gs[j]).axis('off')

fig.suptitle("Trajectories – BlueCylinder_BlueGreenCylinder", y=.92, fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
get_left_right_goals(70, radius=60)

In [ ]:
# ------------------------------------------------------------
# Preferred-goal reach vs platform angle   (all two-goal configs)
# ------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportion_confint   # pip install statsmodels

# ── 1.  Merge goal info into movement dataframe ──────────────────────────
# (uncomment if you haven’t created df_joined yet)
# df_joined = pd.merge(
#     df_cut,
#     results_df[['UniqueTrialID', 'FirstReachedGoal']],
#     on='UniqueTrialID',
#     how='left'
# )

# ensure Angle is numeric
df_joined['Angle'] = pd.to_numeric(df_joined['Angle'], errors='coerce')

# ── 2.  Per-config preferred-side counts ─────────────────────────────────
records = []

for cfg, g in df_joined.groupby('ConfigFile'):

    # skip single-goal layouts if you keep such a list
    if 'center_only_configs' in globals() and cfg in center_only_configs:
        continue

    # ---- goal counts for THIS config ------------------------------------
    res   = results_df.loc[results_df['UniqueTrialID'].isin(g['UniqueTrialID'])]
    left  = (res['FirstReachedGoal'] == 'left').sum()
    right = (res['FirstReachedGoal'] == 'right').sum()
    n     = left + right
    if n == 0:
        continue

    # ---- which platform is on the left / right? -------------------------
    if   "BlueGreenCylinder_BlueCylinder" in cfg or "BlueGreen_Blue" in cfg:
        label, colour, pref_cnt = 'BG→B', 'tab:blue',  left   # preferred = Left
    elif "BlueCylinder_BlueGreenCylinder" in cfg or "Blue_BlueGreen" in cfg:
        label, colour, pref_cnt = 'B→BG', 'tab:green', right  # preferred = Right
    else:
        label, colour, pref_cnt = 'other', 'tab:orange', left # symmetrical

    # ---- angle ----------------------------------------------------------
    angle_vals = g['Angle'].dropna().unique()
    if len(angle_vals) != 1:
        print(f'⚠️  Angle ambiguous → skipped: {cfg}')
        continue
    angle = float(angle_vals[0])

    records.append(dict(
        angle=angle, label=label, colour=colour,
        pref_cnt=pref_cnt, n_trials=n
    ))

# ── 3.  Aggregate over *all* configs that share the same angle ───────────
df  = pd.DataFrame(records)
agg = (df.groupby(['angle', 'label', 'colour'], as_index=False)
         .agg(pref_cnt=('pref_cnt', 'sum'),
              n_trials =('n_trials', 'sum')))

# Wilson CIs & preferred-goal proportion
lo, hi = [], []
for s, n in zip(agg['pref_cnt'], agg['n_trials']):
    l, h = proportion_confint(s, n, alpha=.05, method='wilson')
    lo.append(l); hi.append(h)

agg['p_pref']  = agg['pref_cnt'] / agg['n_trials']
agg['lo_pref'] = lo
agg['hi_pref'] = hi
# ------------------------------------------------------------------------

# ── 4.  Plot ─────────────────────────────────────────────────────────────
label_map = {'BG→B': 'Left', 'B→BG': 'Right', 'other': 'Symmetric'}

plt.figure(figsize=(6, 4))

for lbl, grp in agg.groupby('label'):
    plt.scatter(grp['angle'], grp['p_pref'],
                c=grp['colour'],
                label=label_map[lbl],
                s=60, alpha=.9)

    grp = grp.sort_values('angle')
    plt.plot(grp['angle'], grp['p_pref'],
             c=grp['colour'].iloc[0], lw=1, alpha=.4)

    plt.errorbar(grp['angle'], grp['p_pref'],
                 yerr=[grp['p_pref'] - grp['lo_pref'],
                       grp['hi_pref'] - grp['p_pref']],
                 fmt='none',
                 ecolor=grp['colour'].iloc[0],
                 alpha=.5, capsize=3)

plt.axhline(.5, ls='--', alpha=.4)
plt.xlabel('Platform angle (deg)')
plt.ylabel('Fraction of runs reaching preferred goal')
plt.title('Preferred-goal reach vs. platform angle   (95 % Wilson CIs)')
plt.legend(title='Preferred Goal')
plt.tight_layout()
plt.show()